In [ ]:

from google.colab import output
import multiprocessing, os


class MedriSA(BaseWorker):
    def __init__(self, name, state):
        super().__init__(name, state)
        self.files = ['path.txt']
    
    def working(self):
        # Init
        if not os.path.isfile('/usr/bin/rclone'):
            print('----- Installing rclone -----')
            !curl https://rclone.org/install.sh | sudo bash
            output.clear()
        # Get paras
        content = self.read_file('path.txt')
        if len(content) < 4:
            print('PARA corrupted')
            self.endmsg = '[MedriSA] Invalid parameters'
            self.success = False
            return
        else:
            self.from_path = content[0]
            self.to_path = content[1]
            self.sa_start = int(content[2])
            self.sa_end = int(content[3])
        # download files
        sa_list = []
        raw_list = range(self.sa_start, self.sa_end + 1)
        for count in raw_list:
            if self.download_file(f'{count}.conf') and self.download_file(f'{count}.txt'):
                sa_list.append(count)
        print(f'Downloading completed. {len(sa_list)}/{len(raw_list)} SAs loaded.')
        # starting
        print('----- START -----')
        p = multiprocessing.Pool(len(sa_list))
        p.map(self.start_sa, sa_list)
        p.close()
        p.join()
        # completed
        print('----- DONE -----')
        self.endmsg = f'{len(sa_list)}/{len(raw_list)} SAs completed.'

    
    def start_sa(self, count):
        print(f'Starting SA {count} ...')
        get_ipython().system_raw(f"rclone --config=/content/{count}.conf copy 'FROM:{self.from_path}' 'TO:{self.to_path}' --files-from {count}.txt --no-check-certificate --drive-acknowledge-abuse --drive-server-side-across-configs -v")
        print(f'SA {count} completed')
            
